# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [57]:
import numpy as np
import pandas as pd
from scipy.stats import percentileofscore as score
import math
import xlsxwriter
import requests

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [58]:
stocks =pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [59]:
from secrets import IEX_CLOUD_API_TOKEN

In [60]:
symbol= 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2232357571275,
 'week52high': 137.87,
 'week52low': 57.23,
 'week52change': 0.8475287344412654,
 'sharesOutstanding': 17772758107,
 'float': 0,
 'avg10Volume': 98811875,
 'avg30Volume': 97402435,
 'day200MovingAvg': 112.9,
 'day50MovingAvg': 123.9,
 'employees': 0,
 'ttmEPS': 3.39,
 'ttmDividendRate': 0.8309236745183758,
 'dividendYield': 0.006586363722011468,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-22',
 'nextEarningsDate': '0',
 'peRatio': 39.19559222944944,
 'beta': 1.1954280447343735,
 'maxChangePercent': 48.29189777644029,
 'year5ChangePercent': 4.1196870604912075,
 'year2ChangePercent': 2.2178208917049265,
 'year1ChangePercent': 0.9105846619198806,
 'ytdChangePercent': 0.78603787504027,
 'month6ChangePercent': 0.505228127788474,
 'month3ChangePercent': 0.11200224769112462,
 'month1ChangePercent': 0.07352602158527716,
 'day30ChangePercent': 0.07392580017537696,
 'day5ChangePercent': 0.02928211022813038}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [61]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [62]:
final_dataframe = pd.DataFrame(columns = my_columns)


for symbol_string in symbol_strings:
    batch_api = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data= requests.get(batch_api).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ], index= my_columns), ignore_index= True)
final_dataframe
    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,120.010,0.429191,N/A
1,AAL,17.620,-0.381647,N/A
2,AAP,167.430,0.0367421,N/A
3,AAPL,127.808,0.886978,N/A
4,ABBV,105.190,0.243342,N/A
5,ABC,98.850,0.196562,N/A
6,ABMD,288.000,0.533173,N/A
7,ABT,107.480,0.269195,N/A
8,ACN,259.200,0.218472,N/A
9,ADBE,492.379,0.538202,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [63]:
final_dataframe.sort_values('One-Year Price Return', ascending= False, inplace= True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop= True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,37.900,2.14132,N/A
1,NVDA,555.720,1.44438,N/A
2,AMD,97.630,1.38791,N/A
3,LB,39.860,1.25802,N/A
4,ALB,145.000,1.12338,N/A
5,PYPL,230.940,1.09913,N/A
6,ALGN,515.250,0.975033,N/A
7,NOW,543.900,0.968241,N/A
8,FCX,25.250,0.915026,N/A
9,AAPL,127.808,0.886978,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [64]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Input the portfolio size you wish to invest:")
    
    try:
        portfolio_size = float(portfolio_size)
    except ValueError:
        print("This is not a number. \nplease try again")
        portfolio_size = input("Input the portfolio size you wish to invest:")
        portfolio_size = float(portfolio_size)

        
        

In [65]:
portfolio_input()
portfolio_size

Input the portfolio size you wish to invest:1000000


1000000.0

In [76]:
position_size = portfolio_size/len(final_dataframe)
for i in range(len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,37.900,2.14132,527
1,NVDA,555.720,1.44438,35
2,AMD,97.630,1.38791,204
3,LB,39.860,1.25802,501
4,ALB,145.000,1.12338,137
5,PYPL,230.940,1.09913,86
6,ALGN,515.250,0.975033,38
7,NOW,543.900,0.968241,36
8,FCX,25.250,0.915026,792
9,AAPL,127.808,0.886978,156


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [77]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)


In [78]:
#Batch ApI call


for symbol_string in symbol_strings:
    batch_api = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data= requests.get(batch_api).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series([
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                     data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                     data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                     data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                     data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
        ], index= hqm_columns), ignore_index= True)
hqm_dataframe



,Ticker,Price,Number of Shares,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.800,N/A,0.43281,N/A,0.364598,N/A,0.176426,N/A,0.0676463,N/A,N/A
1,AAL,17.590,N/A,-0.389455,N/A,0.0194284,N/A,0.295915,N/A,0.403963,N/A,N/A
2,AAP,165.030,N/A,0.0382202,N/A,0.163299,N/A,0.0214006,N/A,0.037951,N/A,N/A
3,AAPL,129.303,N/A,0.892105,N/A,0.496765,N/A,0.109722,N/A,0.0733726,N/A,N/A
4,ABBV,104.321,N/A,0.240927,N/A,0.138702,N/A,0.147432,N/A,0.0398564,N/A,N/A
5,ABC,100.520,N/A,0.189523,N/A,0.0383213,N/A,0.0485548,N/A,-0.0859725,N/A,N/A
6,ABMD,287.000,N/A,0.538001,N/A,0.120301,N/A,-0.0141943,N/A,0.0122467,N/A,N/A
7,ABT,108.600,N/A,0.265835,N/A,0.211217,N/A,0.0117003,N/A,-0.0472634,N/A,N/A
8,ACN,253.512,N/A,0.212867,N/A,0.254726,N/A,0.032742,N/A,0.0227446,N/A,N/A
9,ADBE,492.830,N/A,0.519165,N/A,0.203725,N/A,-0.031389,N/A,0.0286767,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [79]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

In [80]:
time_period =[
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month']

for row in hqm_dataframe.index:
    for period in time_period:
        change_col =f'{period} Price Return'
        percentile_col = f'{period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],hqm_dataframe.loc[row,change_col])   
hqm_dataframe

,Ticker,Price,Number of Shares,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.800,N/A,0.43281,87.3267,0.364598,77.0297,0.176426,60.396,0.0676463,64.7525,N/A
1,AAL,17.590,N/A,-0.389455,2.77228,0.0194284,13.0693,0.295915,84.5545,0.403963,98.8119,N/A
2,AAP,165.030,N/A,0.0382202,41.3861,0.163299,38.4158,0.0214006,22.5743,0.037951,51.8812,N/A
3,AAPL,129.303,N/A,0.892105,98.2178,0.496765,89.3069,0.109722,43.3663,0.0733726,66.7327,N/A
4,ABBV,104.321,N/A,0.240927,70.495,0.138702,33.4653,0.147432,52.0792,0.0398564,52.8713,N/A
5,ABC,100.520,N/A,0.189523,64.1584,0.0383213,16.2376,0.0485548,29.703,-0.0859725,3.36634,N/A
6,ABMD,287.000,N/A,0.538001,92.6733,0.120301,30.495,-0.0141943,13.2673,0.0122467,38.8119,N/A
7,ABT,108.600,N/A,0.265835,74.8515,0.211217,49.1089,0.0117003,20,-0.0472634,9.50495,N/A
8,ACN,253.512,N/A,0.212867,67.3267,0.254726,60.5941,0.032742,25.9406,0.0227446,44.3564,N/A
9,ADBE,492.830,N/A,0.519165,92.0792,0.203725,47.9208,-0.031389,10.297,0.0286767,47.3267,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [81]:
from statistics import mean 

for row in hqm_dataframe.index:
    momentum_percntiles_mean = []
    for period in time_periods:
        momentum_percntiles_mean.append(hqm_dataframe.loc[row,f'{period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percntiles_mean)
hqm_dataframe

,Ticker,Price,Number of Shares,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.800,N/A,0.43281,87.3267,0.364598,77.0297,0.176426,60.396,0.0676463,64.7525,72.3762
1,AAL,17.590,N/A,-0.389455,2.77228,0.0194284,13.0693,0.295915,84.5545,0.403963,98.8119,49.802
2,AAP,165.030,N/A,0.0382202,41.3861,0.163299,38.4158,0.0214006,22.5743,0.037951,51.8812,38.5644
3,AAPL,129.303,N/A,0.892105,98.2178,0.496765,89.3069,0.109722,43.3663,0.0733726,66.7327,74.4059
4,ABBV,104.321,N/A,0.240927,70.495,0.138702,33.4653,0.147432,52.0792,0.0398564,52.8713,52.2277
5,ABC,100.520,N/A,0.189523,64.1584,0.0383213,16.2376,0.0485548,29.703,-0.0859725,3.36634,28.3663
6,ABMD,287.000,N/A,0.538001,92.6733,0.120301,30.495,-0.0141943,13.2673,0.0122467,38.8119,43.8119
7,ABT,108.600,N/A,0.265835,74.8515,0.211217,49.1089,0.0117003,20,-0.0472634,9.50495,38.3663
8,ACN,253.512,N/A,0.212867,67.3267,0.254726,60.5941,0.032742,25.9406,0.0227446,44.3564,49.5545
9,ADBE,492.830,N/A,0.519165,92.0792,0.203725,47.9208,-0.031389,10.297,0.0286767,47.3267,49.4059


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [84]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace= True)
hqm_dataframe.reset_index(drop=True, inplace= True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe

/home/edwin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Ticker,Price,Number of Shares,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,24.362,N/A,0.895167,98.4158,1.36722,99.802,0.503553,96.2376,0.209042,90.8911,96.3366
1,LRCX,498.250,N/A,0.788903,96.8317,0.727852,96.6337,0.614345,98.2178,0.167551,87.1287,94.703
2,ALGN,520.710,N/A,0.975332,98.8119,1.00002,99.2079,0.543869,97.6238,0.13093,82.3762,94.505
3,XLNX,157.520,N/A,0.615663,94.4554,0.704037,95.8416,0.49923,95.8416,0.200019,89.703,93.9604
4,AMAT,91.370,N/A,0.500744,90.495,0.575847,94.0594,0.567064,97.8218,0.227843,93.2673,93.9109
5,LB,40.870,N/A,1.30108,99.4059,1.60137,100,0.35915,90.6931,0.152122,85.3465,93.8614
6,TWTR,55.710,N/A,0.770317,96.4356,0.567924,93.8614,0.368051,91.6832,0.223207,92.8713,93.7129
7,ALB,143.380,N/A,1.14262,99.2079,0.842239,98.2178,0.423444,93.6634,0.131247,82.5743,93.4158
8,MU,74.500,N/A,0.437791,87.7228,0.507745,90.8911,0.501736,96.0396,0.266883,96.0396,92.6733
9,URI,245.890,N/A,0.484676,90.099,0.633884,95.2475,0.360187,91.0891,0.155058,85.9406,90.5941


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [94]:
porfolio_size =portfolio_input()



Input the portfolio size you wish to invest:1000000


In [102]:
position_size =(portfolio_size/ len(hqm_dataframe))
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, 'Number of Shares'] = math.floor(position_size /hqm_dataframe.loc[row, 'Price'])
hqm_dataframe
    

,Ticker,Price,Number of Shares,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,24.362,820,0.895167,98.4158,1.36722,99.802,0.503553,96.2376,0.209042,90.8911,96.3366
1,LRCX,498.250,40,0.788903,96.8317,0.727852,96.6337,0.614345,98.2178,0.167551,87.1287,94.703
2,ALGN,520.710,38,0.975332,98.8119,1.00002,99.2079,0.543869,97.6238,0.13093,82.3762,94.505
3,XLNX,157.520,126,0.615663,94.4554,0.704037,95.8416,0.49923,95.8416,0.200019,89.703,93.9604
4,AMAT,91.370,218,0.500744,90.495,0.575847,94.0594,0.567064,97.8218,0.227843,93.2673,93.9109
5,LB,40.870,489,1.30108,99.4059,1.60137,100,0.35915,90.6931,0.152122,85.3465,93.8614
6,TWTR,55.710,359,0.770317,96.4356,0.567924,93.8614,0.368051,91.6832,0.223207,92.8713,93.7129
7,ALB,143.380,139,1.14262,99.2079,0.842239,98.2178,0.423444,93.6634,0.131247,82.5743,93.4158
8,MU,74.500,268,0.437791,87.7228,0.507745,90.8911,0.501736,96.0396,0.266883,96.0396,92.6733
9,URI,245.890,81,0.484676,90.099,0.633884,95.2475,0.360187,91.0891,0.155058,85.9406,90.5941


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [112]:
writer = pd.ExcelWriter('momentum_stategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [113]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [114]:
column_formats = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template]
    
}
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()

## Saving Our Excel Output

As before, saving our Excel output is very easy: